# YOLOv11 OBB Model Training - Google Colab

This notebook provides a complete workflow for training, validating, and running inference with YOLOv11 Oriented Bounding Box (OBB) models in Google Colab.

**Prerequisites:**
- Google Colab GPU access (recommended)
- A data.yaml file with your OBB dataset configuration
- Training images with oriented bounding box annotations

**Features:**
- Install and manage dependencies
- Load pretrained YOLOv11 OBB models
- Train models with configurable hyperparameters
- Validate trained models
- Run inference on images/videos
- Visualize results

## Section 1: Install and Import Dependencies

Install the ultralytics library and import necessary modules for YOLOv11 OBB training in Colab.

In [ ]:
# Install ultralytics library
!pip install -q ultralytics opencv-python pillow matplotlib

# Import required libraries
from ultralytics import YOLO
from pathlib import Path
import os
from IPython.display import Image, display
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import numpy as np

print("✓ Dependencies installed successfully!")
print("GPU Available:", os.system("nvidia-smi -L") == 0)

## Section 2: Load Pretrained YOLOv11 OBB Model

Load a pretrained YOLOv11 OBB model variant. Choose from:
- `yolov11n-obb` - Nano (smallest, fastest)
- `yolov11s-obb` - Small
- `yolov11m-obb` - Medium
- `yolov11l-obb` - Large
- `yolov11x-obb` - Extra Large (largest, most accurate)

In [ ]:
# Configure model variant
MODEL_VARIANT = "yolov11n-obb"  # Change to yolov11s-obb, yolov11m-obb, etc. for larger models

# Load a pretrained YOLOv11 OBB model
print(f"Loading pretrained {MODEL_VARIANT} model...")
model = YOLO(f"{MODEL_VARIANT}.pt")

print("✓ Model loaded successfully!")
print(f"Model: {MODEL_VARIANT}")
print(f"Model architecture: {model.model}")

## Section 3: Prepare Dataset

**Important:** Upload your data.yaml file and ensure your OBB dataset is accessible. Your data.yaml should look like:

```yaml
path: /path/to/dataset
train: images/train
val: images/val
test: images/test
nc: 1  # number of classes
names: ['class_name']  # class names
```

The annotations should be in `.txt` format with OBB format: `<x_center> <y_center> <width> <height> <rotation_angle> <class_id>`

In [ ]:
# Example: If using a public dataset or local file
# For Colab, you can upload files directly or mount Google Drive

# Uncomment to mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Set the path to your data.yaml
DATA_YAML_PATH = "data.yaml"  # Update this path to your dataset

# Verify data.yaml exists
if Path(DATA_YAML_PATH).exists():
    print(f"✓ Found data.yaml at {DATA_YAML_PATH}")
else:
    print(f"⚠ data.yaml not found at {DATA_YAML_PATH}")
    print("Please upload your data.yaml file or update the path above.")

## Section 4: Train YOLOv11 OBB Model

Configure and train the YOLOv11 OBB model with your dataset. Adjust hyperparameters as needed.

In [ ]:
# Configure training parameters
EPOCHS = 100
IMGSZ = 640
BATCH_SIZE = 16
DEVICE = 0  # GPU device (0 for first GPU, -1 for CPU)
PATIENCE = 20  # Early stopping patience
SAVE_DIR = "runs/detect"

print("Training Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Image Size: {IMGSZ}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Device: GPU {DEVICE}" if DEVICE >= 0 else "  Device: CPU")
print(f"  Early Stopping Patience: {PATIENCE}")
print()

# Train the model
print("Starting training...")
results = model.train(
    data=DATA_YAML_PATH,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH_SIZE,
    device=DEVICE,
    patience=PATIENCE,
    save=True,
    project=SAVE_DIR,
    name="obb_model",
    verbose=True,
    # Data augmentation parameters
    augment=True,
    mosaic=1.0,
    flipud=0.5,
    fliplr=0.5,
    degrees=10,
    translate=0.1,
    scale=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    # Performance parameters
    workers=8,
    cache=True,
    close_mosaic=10,
    # Validation parameters
    val=True,
    save_json=False,
    plots=True,
)

print("✓ Training completed!")

## Section 5: Validate Trained Model

Validate the trained model on the validation dataset and display performance metrics.

In [ ]:
# Load the best trained model
BEST_MODEL_PATH = f"{SAVE_DIR}/obb_model/weights/best.pt"

if Path(BEST_MODEL_PATH).exists():
    best_model = YOLO(BEST_MODEL_PATH)
    print(f"✓ Loaded best model from {BEST_MODEL_PATH}")
    
    # Validate the model
    print("\nValidating model on validation set...")
    val_results = best_model.val(
        data=DATA_YAML_PATH,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        device=DEVICE,
        verbose=True,
    )
    
    print("\n✓ Validation completed!")
    print("\nValidation Metrics:")
    print(f"  box_loss: {val_results.box.mean() if hasattr(val_results, 'box') else 'N/A'}")
    print(f"  cls_loss: {val_results.cls.mean() if hasattr(val_results, 'cls') else 'N/A'}")
else:
    print(f"⚠ Best model not found at {BEST_MODEL_PATH}")

## Section 6: Run Inference on Images/Videos

Execute predictions on test images or videos using the trained model.

In [ ]:
# Configure inference parameters
SOURCE_PATH = "path/to/test/image.jpg"  # Change to your image or video path
CONF_THRESHOLD = 0.25  # Confidence threshold for detections

# Example: Using images from the validation set
# Uncomment and modify the paths as needed
# SOURCE_PATH = "runs/detect/obb_model/val/images"

print(f"Running inference on: {SOURCE_PATH}")
print(f"Confidence threshold: {CONF_THRESHOLD}")
print()

# Run inference
if Path(BEST_MODEL_PATH).exists():
    inference_results = best_model.predict(
        source=SOURCE_PATH,
        imgsz=IMGSZ,
        device=DEVICE,
        conf=CONF_THRESHOLD,
        verbose=True,
    )
    
    print("✓ Inference completed!")
    print(f"Number of detections: {len(inference_results)}")
else:
    print("⚠ Model not available. Please train or load a model first.")

## Section 7: Visualize Results

Display predicted bounding boxes, oriented boxes, confidence scores, and comparison plots.

In [ ]:
# Display training results plots
RESULTS_DIR = Path(f"{SAVE_DIR}/obb_model")

if RESULTS_DIR.exists():
    # Display training metrics
    metrics_files = list(RESULTS_DIR.glob("**/results*.csv")) + list(RESULTS_DIR.glob("**/metrics*.png"))
    
    print("Training Results Plots:")
    print("=" * 50)
    
    # Display all .png files from the results directory
    plot_files = list(RESULTS_DIR.rglob("*.png"))
    
    if plot_files:
        for plot_file in sorted(plot_files)[:10]:  # Display first 10 plots
            try:
                print(f"\n📊 {plot_file.name}")
                display(Image(str(plot_file)))
            except Exception as e:
                print(f"Could not display {plot_file.name}: {e}")
    else:
        print("No plots found in results directory.")
else:
    print("⚠ Results directory not found.")

In [ ]:
# Display inference results with detected OBBs
print("\nInference Results:")
print("=" * 50)

if 'inference_results' in locals():
    for i, result in enumerate(inference_results[:5]):  # Display first 5 results
        print(f"\n📷 Result {i+1}")
        
        # Display the result image if available
        if result.plot() is not None:
            # Plot shows detections with boxes
            fig = plt.figure(figsize=(12, 8))
            plt.imshow(result.plot()[..., ::-1])  # Convert BGR to RGB
            plt.axis('off')
            plt.title(f"Detection Result {i+1}")
            plt.tight_layout()
            plt.show()
        
        # Print detection details
        if hasattr(result, 'boxes'):
            print(f"  Detections: {len(result.boxes)}")
            for box in result.boxes:
                if hasattr(box, 'obb'):
                    print(f"    - OBB: {box.obb}, Confidence: {box.conf:.3f}")
                else:
                    print(f"    - Box: {box.xyxy}, Confidence: {box.conf:.3f}")
else:
    print("⚠ No inference results available. Please run inference first.")

## Section 8: Download and Export Results

Export the trained model and results for use outside Colab.

In [ ]:
# For Google Colab: Download trained model and results
import shutil\n\nMODEL_DIR = Path(f"{SAVE_DIR}/obb_model/weights\")\n\nif MODEL_DIR.exists():\n    print(\"Available model files:\")\n    for model_file in MODEL_DIR.glob(\"*.pt\"):\n        file_size = model_file.stat().st_size / (1024 * 1024)  # Convert to MB\n        print(f\"  - {model_file.name} ({file_size:.1f} MB)\")\n    \n    # Create a zip file with all results\n    print(\"\\nCreating zip file with training results...\")\n    try:\n        shutil.make_archive(\n            'obb_model_results',\n            'zip',\n            f\"{SAVE_DIR}/obb_model\"\n        )\n        print(\"✓ Created obb_model_results.zip\")\n        print(\"  You can download this file from Colab Files panel\")\n    except Exception as e:\n        print(f\"Could not create zip: {e}\")\nelse:\n    print(\"⚠ Model directory not found\")

## Section 9: Utility Functions

Helper functions for common tasks.

In [ ]:
def load_model(model_path):\n    \"\"\"\n    Load a trained YOLOv11 OBB model.\n    \n    Args:\n        model_path: Path to the model weights file\n    \n    Returns:\n        Loaded YOLO model\n    \"\"\"\n    if Path(model_path).exists():\n        print(f\"Loading model from {model_path}...\")\n        model = YOLO(model_path)\n        print(\"✓ Model loaded successfully!\")\n        return model\n    else:\n        print(f\"Error: Model not found at {model_path}\")\n        return None\n\n\ndef run_batch_inference(model, image_dir, conf=0.25):\n    \"\"\"\n    Run inference on all images in a directory.\n    \n    Args:\n        model: YOLO model object\n        image_dir: Directory containing images\n        conf: Confidence threshold\n    \n    Returns:\n        List of detection results\n    \"\"\"\n    print(f\"Running inference on images in {image_dir}...\")\n    results = model.predict(\n        source=image_dir,\n        imgsz=IMGSZ,\n        device=DEVICE,\n        conf=conf,\n        verbose=True\n    )\n    print(f\"✓ Inference completed! Processed {len(results)} images\")\n    return results\n\n\ndef export_model(model, export_format=\"onnx\"):\n    \"\"\"\n    Export the model to different formats.\n    \n    Supported formats: torchscript, onnx, openvino, tflite, pb, saved_model, rknn\n    \n    Args:\n        model: YOLO model object\n        export_format: Export format\n    \n    Returns:\n        Path to exported model\n    \"\"\"\n    print(f\"Exporting model to {export_format.upper()}...\")\n    try:\n        exported_path = model.export(format=export_format)\n        print(f\"✓ Model exported to {exported_path}\")\n        return exported_path\n    except Exception as e:\n        print(f\"Error exporting model: {e}\")\n        return None\n\nprint(\"✓ Utility functions defined successfully!\")